In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

In [11]:

def url_parser(sDate, eDate, row=10):
    key="VFYXI%2F3A5kIk7QVsO1epAlZSb6ZtCO9N5VC%2FMF3pW6oZ7%2BsFrvH1Y9y0UfmVzTnA0k2FasoNM6t6JsY7%2BEwNEw%3D%3D"

    # 기본 url + 사용자 key
    url="http://api.visitkorea.or.kr/openapi/service/rest/KorService/searchFestival?serviceKey=" + key
    url=url+"&MobileOS=ETC&MobileApp=AppTest&arrange=A&listYN=Y"   # 기본 필수 옵션
    url=url+"&numOfRows="+ str(row)   # 한페이지 출력 데이터 개수
    url=url+"&eventStartDate="+str(sDate)  # 조회 시작일
    url=url+"&eventEndDate="+str(eDate)

    xml_soup = requests.get(url)
    if xml_soup.status_code != 200:
        print('API를 연결하지 못했습니다.')
        exit()
    
    soup=bs(xml_soup.content, 'html.parser')
    return soup
def tour_info(sDate, eDate, soup):
    ### 파씽한 데이터에서 원하는 데이터 추출 및 DataFrame로 변경
    row=soup.find('totalcount').text
    soup=url_parser(sDate, eDate, row)
    
    items=soup.find_all('item')
    df_list=[]
    
    for item in items:
        staD=item.find('eventstartdate').text      # 축제 시작일
        endD=item.find('eventenddate').get_text()  # 축제 종료일
        titD=item.find('title').get_text()         # 축제명
        try:
            addD=item.find('addr1').get_text()         # 주소
        except:
            addD="온라인개최"
        try:
            telD=item.find('tel').get_text()           # 전화
        except:
            telD="-"
        try:
            lngD=item.find('mapx').text                # 경도
            latD=item.find('mapy').get_text()          # 위도
        except:
            lngD=0
            latD=0
        
        df_list.append({"시작일":staD, "종료일":endD, "축제명":titD,
                        "주소":addD, "연락처":telD, '위도':latD, '경도':lngD})
        
    df1=pd.DataFrame(df_list)
    
    return df1

#print(url)
sDate="20210101"
eDate="20211231"
row=10
### API 연결 및 데이터 파씽

soup = url_parser(sDate, eDate, row=10)
print(soup)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>0000</resultcode><resultmsg>OK</resultmsg></header><body><items><item><addr1>충청남도 부여군 규암면 백제문로 388</addr1><areacode>34</areacode><cat1>A02</cat1><cat2>A0208</cat2><cat3>A02080100</cat3><contentid>1258353</contentid><contenttypeid>15</contenttypeid><createdtime>20110418095420</createdtime><eventenddate>20211030</eventenddate><eventstartdate>20210306</eventstartdate><firstimage>http://tong.visitkorea.or.kr/cms/resource/54/2483454_image2_1.JPG</firstimage><firstimage2>http://tong.visitkorea.or.kr/cms/resource/54/2483454_image2_1.JPG</firstimage2><mapx>126.8997016069</mapx><mapy>36.3048428766</mapy><mlevel>6</mlevel><modifiedtime>20210226112411</modifiedtime><readcount>56458</readcount><sigungucode>6</sigungucode><tel>041-832-5765</tel><title>가무악극으로 만나는 토요 상설공연</title></item><item><addr1>경상북도 칠곡군 가산면 학하들안2길 105</addr1><areacode>35</areacode><cat1>A02</cat1><cat2>A0207</cat2><cat3>A02070200</cat3><contentid